<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 12:14:22
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: -16.90 K (-0.12%)
Current PnL: -31.56 L (-19.73%)
CY Booked + Current PnL: -15.88 L (-9.93%)
-------------------
Total profit:  1.29 L
Total loss:  -32.85 L
-------------------
Total Booked + Current PnL: 11.10 L (8.35%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.97%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.90 L (67.82%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.46,3.90,17.00,21.57,17307.0,3825.0,101805.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.56,XR,ATH,FINANCE
18,COALINDIA,-0.49,1.81,15.58,17.67,23865.0,2720.0,153180.0,478.51,21.95,48.0,L-LC,11.31,185.0,0.11,1.11,22.97,XY25,ATH,MINING
77,TTKPRESTIG,-0.26,-27.50,38.05,0.09,27801.0,-27712.0,73065.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.56,OX40N,NTT,DURABLES
51,MEDANTA,0.30,-1.82,26.37,24.08,32869.0,-2305.0,124645.0,1486.00,-2.50,58.0,X-SC,11.01,87.0,-0.07,0.90,14.17,XY24,NTT,HEALTHCARE
36,ICICIGI,0.67,2.82,16.06,19.34,33045.0,5648.0,205757.0,2252.93,-13.46,62.0,X-MC,2.87,66.0,0.17,1.49,19.21,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,8.21,-43.27,156.81,45.70,157299.0,-76503.0,100312.0,26.4,4.06,84.0,M-SC,22.61,204.0,-0.49,0.73,65.27,XY24,NTT,TRAVEL
44,ITC,2.36,-20.23,38.88,10.78,74529.0,-48628.0,191690.0,452.0,-53.09,49.0,X-LC,18.82,1.0,-0.65,1.39,5.58,X40,NTT,FMCG
62,SATIN,2.09,-12.22,72.98,51.83,176984.0,-33774.0,242510.0,274.0,-21.16,59.0,H-SC,4.30,159.0,-0.19,1.76,18.14,XY24,NTT,FINANCE
57,RECLTD,1.90,-1.08,22.06,20.74,44334.0,-2200.0,200970.0,446.0,9.37,54.0,M-MC,3.00,191.0,-0.05,1.46,10.75,XY25,NTT,FINANCE
49,LAOPALA,1.66,-41.39,137.32,39.09,105515.0,-54266.0,76839.0,464.0,67.64,45.0,H-SC,15.32,154.0,-0.51,0.56,2.75,AR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,-2.98,-24.78,113.74,60.77,156364.0,-45300.0,137475.0,521.0,58.68,54.0,H-SC,1.42,160.0,-0.29,1.00,24.64,XR,NTT,JEWELLERY
14,BSOFT,-2.49,-31.79,121.05,50.79,116140.0,-44709.0,95944.0,831.7,-12.64,36.0,H-SC,7.00,172.0,-0.38,0.70,11.71,XR,ATH,IT
21,DABUR,-2.28,-1.27,45.54,43.70,111539.0,-3143.0,244925.0,735.0,-10.42,44.0,X-MC,1.83,70.0,-0.03,1.78,13.59,XY24,BTT,FMCG
0,5PAISA,-2.28,-39.07,85.31,12.90,113565.0,-85376.0,133120.0,593.0,95.91,39.0,H-SC,6.21,170.0,-0.75,0.97,10.69,OX40N,NTT,FINANCE
84,WIPRO,-2.17,-13.45,99.04,72.27,145244.0,-22796.0,146652.0,420.0,-21.74,26.0,M-LC,6.45,98.0,-0.16,1.06,0.00,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.30,4.00,99.91,107.91,153417.0,5911.0,153555.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.52,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.26,-27.50,38.05,0.09,27801.0,-27712.0,73065.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.56,OX40N,NTT,DURABLES
66,SIS,1.03,-28.13,68.72,21.26,54840.0,-31230.0,79802.0,528.00,1877.20,38.0,H-SC,4.22,168.0,-0.57,0.58,10.02,OX40N,NTT,MISC
72,TATAELXSI,-1.11,-29.89,88.32,32.03,81631.0,-39409.0,92426.0,9161.00,-12.30,35.0,H-SC,7.25,158.0,-0.48,0.67,3.52,OX40N,NTT,IT
40,INDIGOPNTS,-0.33,-30.75,44.44,0.02,53717.0,-53684.0,120875.0,1408.00,84.15,30.0,M-SC,8.68,222.0,-1.00,0.88,5.14,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.37,20.11,20.88,45.2,48089.0,38568.0,230310.0,1856.0,37.53,67.0,X-MC,4.48,79.0,0.8,1.67,28.28,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.49,1.81,15.58,17.67,23865.0,2720.0,153180.0,478.51,21.95,48.0,L-LC,11.31,185.0,0.11,1.11,22.97,XY25,ATH,MINING
26,FINCABLES,0.30,4.00,99.91,107.91,153417.0,5911.0,153555.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.52,OX40N,ATH,CABLES
50,MASFIN,0.46,3.90,17.00,21.57,17307.0,3825.0,101805.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.56,XR,ATH,FINANCE
1,ABB,-0.34,6.71,36.42,45.57,101673.0,17549.0,279168.0,7934.00,-31.40,66.0,H-MC,4.29,125.0,0.17,2.02,23.97,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.46,3.90,17.00,21.57,17307.0,3825.0,101805.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.56,XR,ATH,FINANCE
1,ABB,-0.34,6.71,36.42,45.57,101673.0,17549.0,279168.0,7934.00,-31.40,66.0,H-MC,4.29,125.0,0.17,2.02,23.97,AR,NTT,ELECTRICAL
26,FINCABLES,0.30,4.00,99.91,107.91,153417.0,5911.0,153555.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.52,OX40N,ATH,CABLES
39,INDIAMART,-1.58,-5.95,119.80,106.72,138963.0,-7340.0,115996.0,4810.62,-57.62,45.0,H-SC,7.80,140.0,-0.05,0.84,16.04,AR,ATH,MISC
85,ZYDUSLIFE,-0.04,-4.43,41.91,35.63,83942.0,-9282.0,200292.0,1286.17,-16.67,52.0,H-MC,5.43,120.0,-0.11,1.45,11.50,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.49,-6.70,43.89,34.24,128122.0,-20972.0,291916.0,1972.00,-26.71,24.0,X-LC,3.95,3.0,-0.16,2.12,1.02,X40,NTT,IT
73,TCS,-1.26,-26.23,60.69,18.55,169351.0,-99195.0,279042.0,4311.58,-33.46,26.0,X-LC,4.23,2.0,-0.59,2.02,0.00,X40,ATH,IT
33,HCLTECH,1.42,-3.78,28.71,23.83,66828.0,-9156.0,232768.0,1908.19,0.91,29.0,X-LC,4.81,8.0,-0.14,1.69,11.68,X40,ATH,IT
8,AWL,-0.88,-34.45,139.80,57.19,298862.0,-112349.0,213778.0,485.00,-62.47,29.0,X-SC,17.57,81.0,-0.38,1.55,0.34,XY24,NTT,FMCG
13,BERGEPAINT,-0.86,-18.16,48.02,21.14,89345.0,-41282.0,186057.0,680.00,-19.97,29.0,X-MC,12.44,68.0,-0.46,1.35,0.20,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.05,4.66,15.39,20.77,37713.0,10904.0,245048.0,1644.00,-7.24,64.0,X-MC,1.46,64.0,0.29,1.78,12.40,X40N,NTT,BREWERIES
21,DABUR,-2.28,-1.27,45.54,43.70,111539.0,-3143.0,244925.0,735.00,-10.42,44.0,X-MC,1.83,70.0,-0.03,1.78,13.59,XY24,BTT,FMCG
65,SIEMENS,0.08,-14.32,46.48,25.51,74115.0,-26640.0,159455.0,4671.50,33.99,56.0,H-LC,1.95,51.0,-0.36,1.16,17.51,AR,ATH,ELECTRICAL
34,HINDUNILVR,-0.00,-10.05,26.37,13.67,60973.0,-25843.0,231220.0,2922.00,-33.31,41.0,X-LC,2.87,9.0,-0.42,1.68,9.16,XY25,NTT,FMCG
36,ICICIGI,0.67,2.82,16.06,19.34,33045.0,5648.0,205757.0,2252.93,-13.46,62.0,X-MC,2.87,66.0,0.17,1.49,19.21,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.26,-26.23,60.69,18.55,169351.0,-99195.0,279042.0,4311.58,-33.46,26.0,X-LC,4.23,2.0,-0.59,2.02,0.00,X40,ATH,IT
13,BERGEPAINT,-0.86,-18.16,48.02,21.14,89345.0,-41282.0,186057.0,680.00,-19.97,29.0,X-MC,12.44,68.0,-0.46,1.35,0.20,XY24,NTT,PAINTS
8,AWL,-0.88,-34.45,139.80,57.19,298862.0,-112349.0,213778.0,485.00,-62.47,29.0,X-SC,17.57,81.0,-0.38,1.55,0.34,XY24,NTT,FMCG
12,BATAINDIA,-0.75,-46.30,151.77,35.20,104870.0,-59572.0,69098.0,2096.00,-9.46,36.0,X-SC,21.53,91.0,-0.57,0.50,0.44,X40,NTT,FOOTWEAR
3,ACC,0.02,-31.09,138.21,64.15,226623.0,-73981.0,163970.0,3906.00,-48.05,39.0,X-SC,8.48,85.0,-0.33,1.19,0.46,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.12,-30.14,104.48,42.84,47446.0,-19594.0,45412.0,424.00,-54.27,51.0,X-SC,18.85,80.0,-0.41,0.33,9.23,XY24,NTT,MISC
12,BATAINDIA,-0.75,-46.30,151.77,35.20,104870.0,-59572.0,69098.0,2096.00,-9.46,36.0,X-SC,21.53,91.0,-0.57,0.50,0.44,X40,NTT,FOOTWEAR
58,RELAXO,0.15,-52.25,218.96,52.31,151774.0,-75844.0,69316.0,1176.00,-49.18,45.0,X-SC,14.07,92.0,-0.50,0.50,3.92,X40N,NTT,FOOTWEAR
51,MEDANTA,0.30,-1.82,26.37,24.08,32869.0,-2305.0,124645.0,1486.00,-2.50,58.0,X-SC,11.01,87.0,-0.07,0.90,14.17,XY24,NTT,HEALTHCARE
35,HONAUT,-0.11,-23.03,86.21,43.32,108073.0,-37512.0,125360.0,58357.33,-32.50,39.0,X-SC,11.16,90.0,-0.35,0.91,1.79,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,2.36,-20.23,38.88,10.78,74529.0,-48628.0,191690.0,452.00,-53.09,49.0,X-LC,18.82,1.0,-0.65,1.39,5.58,X40,NTT,FMCG
73,TCS,-1.26,-26.23,60.69,18.55,169351.0,-99195.0,279042.0,4311.58,-33.46,26.0,X-LC,4.23,2.0,-0.59,2.02,0.00,X40,ATH,IT
42,INFY,-1.49,-6.70,43.89,34.24,128122.0,-20972.0,291916.0,1972.00,-26.71,24.0,X-LC,3.95,3.0,-0.16,2.12,1.02,X40,NTT,IT
75,TMPV,-0.84,-22.19,58.04,22.98,102022.0,-50120.0,175778.0,600.00,-82.53,60.0,X-LC,2.96,4.0,-0.49,1.27,12.37,XY24,NTT,AUTO
33,HCLTECH,1.42,-3.78,28.71,23.83,66828.0,-9156.0,232768.0,1908.19,0.91,29.0,X-LC,4.81,8.0,-0.14,1.69,11.68,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.16,-30.43,91.02,32.89,48888.0,-23496.0,53711.0,1800.00,-352.80,64.0,L-MC,11.12,258.0,-0.48,0.39,47.92,XR,NTT,BANKS
67,SONACOMS,-0.41,-8.73,52.39,39.09,48372.0,-8829.0,92330.0,804.02,-27.63,62.0,M-MC,6.66,193.0,-0.18,0.67,30.62,AR,ATH,AUTO
25,EASEMYTRIP,8.21,-43.27,156.81,45.70,157299.0,-76503.0,100312.0,26.40,4.06,84.0,M-SC,22.61,204.0,-0.49,0.73,65.27,XY24,NTT,TRAVEL
50,MASFIN,0.46,3.90,17.00,21.57,17307.0,3825.0,101805.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.56,XR,ATH,FINANCE
18,COALINDIA,-0.49,1.81,15.58,17.67,23865.0,2720.0,153180.0,478.51,21.95,48.0,L-LC,11.31,185.0,0.11,1.11,22.97,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,8.21,-43.27,156.81,45.70,157299.0,-76503.0,100312.0,26.40,4.06,84.0,M-SC,22.61,204.0,-0.49,0.73,65.27,XY24,NTT,TRAVEL
7,ATULAUTO,-0.47,-11.67,64.80,45.57,112837.0,-23001.0,174131.0,844.00,3775.68,68.0,M-SC,4.43,248.0,-0.20,1.26,31.78,XY24,NTT,AUTO
11,BANDHANBNK,1.50,-17.76,133.97,92.42,306907.0,-49473.0,229086.0,400.00,96.84,71.0,H-SC,7.04,174.0,-0.16,1.66,32.39,XY24,NTT,BANKS
1,ABB,-0.34,6.71,36.42,45.57,101673.0,17549.0,279168.0,7934.00,-31.40,66.0,H-MC,4.29,125.0,0.17,2.02,23.97,AR,NTT,ELECTRICAL
67,SONACOMS,-0.41,-8.73,52.39,39.09,48372.0,-8829.0,92330.0,804.02,-27.63,62.0,M-MC,6.66,193.0,-0.18,0.67,30.62,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.18
1,25,44.69
2,50,76.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.42
MC    30.37
LC    23.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.94
X40      23.17
X40N     15.20
AR        8.99
XY25      8.84
XR        8.70
OX40N     7.30
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.90
H-SC    22.71
X-LC    19.89
X-SC    11.86
M-SC    11.06
H-MC     4.95
M-MC     2.13
H-LC     1.16
L-LC     1.11
M-LC     1.06
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.01,-14.44,52.89
FINANCE,13.48,-12.37,56.59
IT,11.46,-35.27,102.77
MISC,7.07,-33.39,87.96
ELECTRICAL,6.28,-8.45,48.02
PAINTS,5.08,-31.86,51.45
INSURANCE,4.89,-0.50,34.90
PHARMA,4.11,-3.71,36.36
AUTO,3.20,-18.53,59.52


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3256516.0,21
AR,1331507.0,10
X40,1300620.0,15
XR,1291648.0,12
X40N,1025361.0,10
OX40N,785592.0,10
XY25,399596.0,6
SR,299116.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3662467.0,24
M-SC,1392102.0,14
X-MC,1356587.0,15
X-SC,1168415.0,10
X-LC,1147367.0,12
H-MC,395870.0,3
L-SC,182330.0,2
M-LC,145244.0,1
M-MC,92706.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233446.0      6
           AR         949389.0      5
           XR         831815.0      7
M-SC       XY24       797824.0      6
X-SC       XY24       605800.0      4
X-LC       X40        584167.0      6
X-MC       X40        539917.0      7
           X40N       413004.0      4
X-SC       X40N       386079.0      4
H-SC       OX40N      348701.0      4
M-SC       OX40N      343745.0      5
H-SC       SR         299116.0      2
X-LC       X40N       226278.0      2
H-MC       XY24       210255.0      1
X-LC       XY24       208307.0      2
X-MC       XY25       202782.0      2
           XY24       200884.0      2
H-MC       AR         185615.0      2
X-SC       X40        176536.0      2
M-SC       XR         176517.0      2
M-LC       XR         145244.0      1
X-LC       XY25       128615.0      2
L-SC       OX40N       93146.0      1
           XR          89184.0      1
H-LC       AR          74115.0      1
M-SC       AR          74016.0      1
L-MC       XR          48888.0      1
M-MC       AR          48372.0      1
           XY25        44334.0      1
L-LC       XY25        23865.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
